# Import Libraries

In [1]:

import yfinance as yf
import pandas as pd
import numpy as np
from typing import Dict, Optional
from tqdm import tqdm

from datetime import datetime

# Define Functions

In [2]:
def get_tickers(url):
    """Fetches base data from a given URL and returns it as a DataFrame.
    Args:
        url (str): The URL to fetch the data from.
    Returns:
        pd.DataFrame: A DataFrame containing the fetched data.
    """
    
    df = pd.read_html(url)[0]
    df = df.rename(columns={'Symbol': 'Ticker', 'Security': 'Company_Name', 'GICS Sector': 'Sector', 'GICS Sub-Industry': 'Industry', 'Founded': 'Founded_Year'}).drop(['Date added', 'CIK'], axis=1)
    df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_')
    df['Ticker'] = df['Ticker'].str.upper()
    
    print(f"Fetched {len(df)} rows from {url}")
    return df['Ticker'].unique().tolist()

In [3]:
def get_stock_data(ticker_symbol: str, period: str = "5y", benchmark_data: Optional[pd.DataFrame] = None) -> Dict:
    """
    Calculate comprehensive stock metrics for a given ticker.
    
    Args:
        ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL')
        period (str): Historical data period (default: '5y')
        benchmark_data (pd.DataFrame): Pre-loaded S&P 500 data for beta calculation
        
    Returns:
        Dict: Dictionary containing all calculated metrics for the ticker
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        # Get historical data
        hist = ticker.history(period=period)
        if hist.empty:
            raise ValueError(f"No data found for {ticker_symbol}")
        
        hist = hist.reset_index()
        hist['Date'] = pd.to_datetime(hist['Date'])
        hist['Year'] = hist['Date'].dt.year
        
        # Get ticker info
        info = ticker.info
        current_year = hist['Year'].max()
        
        # Initialize results dictionary
        stock_data = {'Ticker': ticker_symbol}
        
        # Price-based metrics
        stock_data.update(_calculate_price_metrics(hist))
        
        # Return calculations
        stock_data.update(_calculate_returns(hist, current_year))
        
        # Risk metrics (pass benchmark_data to avoid reloading)
        stock_data.update(_calculate_risk_metrics(hist, benchmark_data))
        
        # Market data
        stock_data.update(_get_market_data(info))
        
        return stock_data
        
    except Exception as e:
        print(f"Error processing {ticker_symbol}: {e}")

def _calculate_price_metrics(hist: pd.DataFrame) -> Dict:
    """Calculate price-related metrics."""
    closing_price = hist['Close'].iloc[-1]
    all_time_high = hist['Close'].max()
    
    # Calculate moving average
    hist['200_Day_Moving_Average'] = hist['Close'].rolling(window=200).mean()
    
    return {
        'Closing_Price': round(closing_price, 2),
        'All_Time_High': round(all_time_high, 2),
        'Percent_From_All_Time_High': round(((closing_price - all_time_high) / all_time_high) * 100, 2),
        'Percent_Difference_200_Day_Moving_Average': round(((closing_price - hist['200_Day_Moving_Average'].iloc[-1]) / hist['200_Day_Moving_Average'].iloc[-1]) * 100, 2),
        '24_Hour_Percent_Change': round(hist['Close'].pct_change(periods=1).iloc[-1] * 100, 2),
        '7_Day_Percent_Change': round(hist['Close'].pct_change(periods=7).iloc[-1] * 100, 2),
        '30_Day_Percent_Change': round(hist['Close'].pct_change(periods=30).iloc[-1] * 100, 2)
    }


def _calculate_returns(hist: pd.DataFrame, current_year: int) -> Dict:
    """Calculate return metrics."""
    returns_data = {}
    
    # Annualized return
    total_years = len(hist['Year'].unique())
    if total_years > 1:
        annualized_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) ** (1 / total_years) - 1) * 100
        returns_data['Annualized_Return'] = round(annualized_return, 2)
    
    # Yearly returns
    for year in sorted(hist['Year'].unique(), reverse=True):
        year_data = hist[hist['Year'] == year]
        if len(year_data) > 1:
            year_return = ((year_data['Close'].iloc[-1] - year_data['Close'].iloc[0]) / year_data['Close'].iloc[0]) * 100
            
            if year == current_year:
                returns_data['YTD_Return'] = round(year_return, 2)
            else:
                returns_data[f'{year}_Return'] = round(year_return, 2)
    
    return returns_data


def _calculate_risk_metrics(hist: pd.DataFrame, benchmark_data: Optional[pd.DataFrame] = None) -> Dict:
    """Calculate risk-related metrics."""
    # Daily returns
    hist['Daily_Return'] = hist['Close'].pct_change().dropna()
    
    # Volatility (annualized)
    volatility = hist['Daily_Return'].std() * np.sqrt(252)
    
    # Sharpe ratio (assuming 1% risk-free rate)
    risk_free_rate = 0.01
    if len(hist) > 1:
        total_return = (hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1
        annualized_return = (1 + total_return) ** (252 / len(hist)) - 1
        excess_return = annualized_return - risk_free_rate
        sharpe_ratio = excess_return / volatility if volatility > 0 else 0
    else:
        sharpe_ratio = 0
    
    # Beta calculation (use pre-loaded benchmark data if available)
    beta = _calculate_beta(hist, benchmark_data)
    
    return {
        'Annualized_Volatility': round(volatility * 100, 2),
        'Sharpe_Ratio': round(sharpe_ratio, 2),
        'Beta': round(beta, 2) if not np.isnan(beta) else None
    }


def _calculate_beta(hist: pd.DataFrame, benchmark_data: Optional[pd.DataFrame] = None) -> float:
    """Calculate beta against S&P 500 using pre-loaded benchmark data."""
    try:
        if benchmark_data is None:
            return np.nan
            
        # Merge on date
        merged = hist.merge(
            benchmark_data[['Date', 'Daily_Return']], 
            on='Date', 
            suffixes=('', '_Benchmark'),
            how='inner'
        )
        
        if len(merged) < 30:  # Need sufficient data points
            return np.nan
        
        # Calculate beta using numpy for speed
        stock_returns = merged['Daily_Return'].dropna()
        benchmark_returns = merged['Daily_Return_Benchmark'].dropna()
        
        if len(stock_returns) == len(benchmark_returns) and len(stock_returns) > 0:
            covariance = np.cov(stock_returns, benchmark_returns)[0, 1]
            benchmark_variance = np.var(benchmark_returns)
            return covariance / benchmark_variance if benchmark_variance != 0 else np.nan
        
        return np.nan
        
    except Exception:
        return np.nan


def _load_benchmark_data(period: str = "5y") -> pd.DataFrame:
    """Load S&P 500 benchmark data once for all calculations."""
    try:
        print("Loading S&P 500 benchmark data...")
        benchmark = yf.Ticker('^GSPC')
        benchmark_hist = benchmark.history(period=period).reset_index()
        benchmark_hist['Date'] = pd.to_datetime(benchmark_hist['Date'])
        benchmark_hist['Daily_Return'] = benchmark_hist['Close'].pct_change()
        return benchmark_hist[['Date', 'Daily_Return']].dropna()
    except Exception as e:
        print(f"Failed to load benchmark data: {e}")
        return pd.DataFrame()


def _get_market_data(info: Dict) -> Dict:
    """Extract market data from ticker info."""
    return {
        'Company_Name': info.get('shortName', '').replace('\n', ' '),
        'Market_Cap': info.get('marketCap'),
        'Sector': info.get('sector'),
        'Industry': info.get('industry'),
        'Country': info.get('country'),
        'Business_Summary': info.get('longBusinessSummary', '').replace('\n', ' '),
        'Dividend_Yield': info.get('dividendYield'),
        'Trailing_PE': info.get('trailingPE'),
        'Forward_PE': info.get('forwardPE'),
        'Average_Volume': info.get('averageVolume'),
        'Average_Volume_10days': info.get('averageVolume10days'),
        '52_Week_Change': info.get('52WeekChange')
    }


def get_multiple_stocks_data(tickers: list, period: str = "5y") -> pd.DataFrame:
    """
    Get stock data for multiple tickers and return as DataFrame.
    
    Args:
        tickers (list): List of ticker symbols
        period (str): Historical data period
        
    Returns:
        pd.DataFrame: DataFrame with stock data for all tickers
    """

    all_data = []

    for ticker in (tickers):
        try:
            stock_data = get_stock_data(ticker)  # your existing function
            if stock_data is not None:  # Only append if data was successfully retrieved
                tqdm.write(f'Processing {ticker}')
                all_data.append(stock_data)
            else:
                tqdm.write(f"Warning: No data retrieved for {ticker}")
        except Exception as e:
            tqdm.write(f"Error processing {ticker}: {str(e)}")
            continue
            
    df = pd.DataFrame(all_data)

    # Clean up Missing Values
    df['Dividend_Yield'] = df['Dividend_Yield'].fillna(0)
    df['Sector'] = df['Sector'].fillna('Unknown')
    df['Industry'] = df['Industry'].fillna('Unknown') 
    df['Country'] = df['Country'].fillna('Unknown')
    df['Business_Summary'] = df['Business_Summary'].fillna('No description available')
    df = df.fillna(0)

    today_date = datetime.now().date().strftime("%Y-%m-%d")
    df['Update_Date'] = today_date

    return df.sort_values('Market_Cap', ascending=False, na_position='last')

# Get List of Tickers

In [4]:
stock_tickers = get_tickers('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
len(stock_tickers)

Fetched 503 rows from https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


503

# Create Dataset with Features

In [5]:
df_enriched_stock_data = get_multiple_stocks_data(stock_tickers)
df_enriched_stock_data

Processing MMM
Processing AOS
Processing ABT
Processing ABBV
Processing ACN
Processing ADBE
Processing AMD
Processing AES
Processing AFL
Processing A
Processing APD
Processing ABNB
Processing AKAM
Processing ALB
Processing ARE
Processing ALGN
Processing ALLE
Processing LNT
Processing ALL
Processing GOOGL
Processing GOOG
Processing MO
Processing AMZN
Processing AMCR
Processing AEE
Processing AEP
Processing AXP
Processing AIG
Processing AMT
Processing AWK
Processing AMP
Processing AME
Processing AMGN
Processing APH
Processing ADI
Processing ANSS
Processing AON
Processing APA
Processing APO
Processing AAPL
Processing AMAT
Processing APTV
Processing ACGL
Processing ADM
Processing ANET
Processing AJG
Processing AIZ
Processing T
Processing ATO
Processing ADSK
Processing ADP
Processing AZO
Processing AVB
Processing AVY
Processing AXON
Processing BKR
Processing BALL
Processing BAC
Processing BAX
Processing BDX


$BRK.B: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


Error processing BRK.B: No data found for BRK.B
Processing BBY
Processing TECH
Processing BIIB
Processing BLK
Processing BX
Processing BK
Processing BA
Processing BKNG
Processing BSX
Processing BMY
Processing AVGO
Processing BR


$BF.B: possibly delisted; no price data found  (period=5y)


Processing BRO
Error processing BF.B: No data found for BF.B
Processing BLDR
Processing BG
Processing BXP
Processing CHRW
Processing CDNS
Processing CZR
Processing CPT
Processing CPB
Processing COF
Processing CAH
Processing KMX
Processing CCL
Processing CARR
Processing CAT
Processing CBOE
Processing CBRE
Processing CDW
Processing COR
Processing CNC
Processing CNP
Processing CF
Processing CRL
Processing SCHW
Processing CHTR
Processing CVX
Processing CMG
Processing CB
Processing CHD
Processing CI
Processing CINF
Processing CTAS
Processing CSCO
Processing C
Processing CFG
Processing CLX
Processing CME
Processing CMS
Processing KO
Processing CTSH
Processing COIN
Processing CL
Processing CMCSA
Processing CAG
Processing COP
Processing ED
Processing STZ
Processing CEG
Processing COO
Processing CPRT
Processing GLW
Processing CPAY
Processing CTVA
Processing CSGP
Processing COST
Processing CTRA
Processing CRWD
Processing CCI
Processing CSX
Processing CMI
Processing CVS
Processing DHR
Processing 

$WBA: possibly delisted; no price data found  (period=5y)


Error processing WBA: No data found for WBA
Processing WMT
Processing DIS
Processing WBD
Processing WM
Processing WAT
Processing WEC
Processing WFC
Processing WELL
Processing WST
Processing WDC
Processing WY
Processing WSM
Processing WMB
Processing WTW
Processing WDAY
Processing WYNN
Processing XEL
Processing XYL
Processing YUM
Processing ZBRA
Processing ZBH
Processing ZTS


/var/folders/rx/jl4f7yr95xd03cgb4sg2w99h0000gn/T/ipykernel_1539/504098595.py:218: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


,Ticker,Closing_Price,All_Time_High,Percent_From_All_Time_High,Percent_Difference_200_Day_Moving_Average,24_Hour_Percent_Change,7_Day_Percent_Change,30_Day_Percent_Change,Annualized_Return,YTD_Return,...,Industry,Country,Business_Summary,Dividend_Yield,Trailing_PE,Forward_PE,Average_Volume,Average_Volume_10days,52_Week_Change,Update_Date
345,NVDA,157.75,157.75,0.00,22.11,1.76,9.46,16.57,60.67,14.07,...,Semiconductors,United States,"NVIDIA Corporation, a computing infrastructure...",0.03,50.887100,38.288837,249322685,187835800,0.254816,2025-06-27
315,MSFT,495.94,497.45,-0.30,17.61,-0.30,3.74,9.69,17.33,18.94,...,Software - Infrastructure,United States,Microsoft Corporation develops and supports so...,0.67,38.355762,33.173244,23145869,20812190,0.112988,2025-06-27
39,AAPL,201.08,258.40,-22.18,-9.79,0.04,2.78,-5.30,14.78,-17.34,...,Consumer Electronics,United States,"Apple Inc. designs, manufactures, and markets ...",0.52,31.369736,24.197351,61975983,51950890,-0.045675,2025-06-27
22,AMZN,223.30,242.06,-7.75,8.96,2.85,3.95,6.21,8.88,1.40,...,Internet Retail,United States,"Amazon.com, Inc. engages in the retail sale of...",0.00,36.368080,36.308945,49333604,39989330,0.123519,2025-06-27
19,GOOGL,178.53,205.89,-13.29,3.89,2.88,1.47,8.09,17.04,-5.53,...,Internet Content & Information,United States,Alphabet Inc. offers various products and plat...,0.48,19.947487,19.925222,41357416,37073840,-0.047269,2025-06-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,IVZ,15.70,24.58,-36.12,-2.56,0.77,8.13,1.42,11.01,-8.82,...,Asset Management,United States,Invesco Ltd. is a publicly owned investment ma...,5.39,12.559999,8.306878,5714119,5047610,0.041444,2025-06-27
37,APA,18.56,46.75,-60.30,-10.65,0.60,-10.68,2.15,7.67,-18.45,...,Oil & Gas E&P,United States,"APA Corporation, an independent energy company...",5.42,6.652329,6.749091,9028964,10513290,-0.373302,2025-06-27
318,MHK,104.90,229.74,-54.34,-15.38,1.50,6.57,-2.20,0.83,-9.51,...,"Furnishings, Fixtures & Appliances",United States,"Mohawk Industries, Inc. designs, manufactures,...",0.00,13.712419,9.408072,803162,920930,-0.090149,2025-06-27
78,CZR,28.86,119.49,-75.85,-14.37,1.23,9.11,-5.96,-5.13,-11.45,...,Resorts & Casinos,United States,"Caesars Entertainment, Inc. operates as a gami...",0.00,0.000000,21.537313,5763859,5734720,-0.282587,2025-06-27


In [6]:
df_enriched_stock_data.columns

Index(['Ticker', 'Closing_Price', 'All_Time_High',
       'Percent_From_All_Time_High',
       'Percent_Difference_200_Day_Moving_Average', '24_Hour_Percent_Change',
       '7_Day_Percent_Change', '30_Day_Percent_Change', 'Annualized_Return',
       'YTD_Return', '2024_Return', '2023_Return', '2022_Return',
       '2021_Return', '2020_Return', 'Annualized_Volatility', 'Sharpe_Ratio',
       'Beta', 'Company_Name', 'Market_Cap', 'Sector', 'Industry', 'Country',
       'Business_Summary', 'Dividend_Yield', 'Trailing_PE', 'Forward_PE',
       'Average_Volume', 'Average_Volume_10days', '52_Week_Change',
       'Update_Date'],
      dtype='object')

In [7]:
def create_summary_chart(df):
    df_result = df.copy()
    
    # Create the 'Risk' column based on the specified conditions
    def determine_Risk_signal(row):
        all_time_high_condition = row['Percent_From_All_Time_High'] < -10
        moving_avg_condition = row['Percent_Difference_200_Day_Moving_Average'] < 0
        
        # If Percent_From_All_Time_High is greater than -10%, then 'High'
        if row['Percent_From_All_Time_High'] >= -10:
            return 'High'
        
        # If both conditions are met (< -10% and negative moving avg), then 'Low'
        if all_time_high_condition and moving_avg_condition:
            return 'Low'
        
        # If only one condition is met, then 'Medium'
        if all_time_high_condition or moving_avg_condition:
            return 'Medium'
        
        # Default case (shouldn't reach here based on logic, but safety)
        return 'High'
    
    # Apply the Risk signal logic
    df_result['Risk'] = df_result.apply(determine_Risk_signal, axis=1)
    
    # Get top 25 by Market Cap first
    df_top_25 = df_result.sort_values('Market_Cap', ascending=False).head(25)
    
    # Create categorical ordering for Risk column
    Risk_order = ['Low', 'Medium', 'High']
    df_top_25['Risk'] = pd.Categorical(df_top_25['Risk'], categories=Risk_order, ordered=True)
    
    # Sort by Risk column (Low, Medium, High), then by Market Cap descending
    df_sorted = df_top_25.sort_values(['Risk', 'Market_Cap'], ascending=[True, False])
    
    # Return the sorted dataframe with selected columns
    return df_sorted[
        ['Ticker', 'Company_Name', 'Market_Cap', 'Closing_Price', 'All_Time_High', 'Risk',
         'Percent_From_All_Time_High', 'Percent_Difference_200_Day_Moving_Average', 
         '24_Hour_Percent_Change', '7_Day_Percent_Change', '30_Day_Percent_Change']
    ].reset_index(drop=True)

In [8]:
create_summary_chart(df_enriched_stock_data)

,Ticker,Company_Name,Market_Cap,Closing_Price,All_Time_High,Risk,Percent_From_All_Time_High,Percent_Difference_200_Day_Moving_Average,24_Hour_Percent_Change,7_Day_Percent_Change,30_Day_Percent_Change
0,AAPL,Apple Inc.,3003290746880,201.08,258.40,Low,-22.18,-9.79,0.04,2.78,-5.30
1,LLY,Eli Lilly and Company,696149409792,775.45,954.83,Low,-18.79,-5.14,-2.47,-2.00,8.59
2,XOM,Exxon Mobil Corporation,471388422144,109.38,122.12,Low,-10.43,-0.68,-0.55,-4.05,1.76
3,HD,"Home Depot, Inc. (The)",366869741568,368.74,426.00,Low,-13.44,-3.65,1.44,5.72,-0.48
4,ABBV,AbbVie Inc.,322032369664,182.31,214.68,Low,-15.08,-2.08,-2.40,-1.71,2.74
5,GOOGL,Alphabet Inc.,2171406909440,178.53,205.89,Medium,-13.29,3.89,2.88,1.47,8.09
6,GOOG,Alphabet Inc.,2157191888896,178.27,207.22,Medium,-13.97,2.79,2.20,0.59,7.00
7,TSLA,"Tesla, Inc.",1042399297536,323.63,479.86,Medium,-32.56,3.45,-0.66,2.30,-6.92
8,NVDA,NVIDIA Corporation,3847144079360,157.75,157.75,High,0.00,22.11,1.76,9.46,16.57
9,MSFT,Microsoft Corporation,3686094077952,495.94,497.45,High,-0.30,17.61,-0.30,3.74,9.69


# Save to CSV

In [9]:
df_enriched_stock_data.isna().sum()

Ticker                                       0
Closing_Price                                0
All_Time_High                                0
Percent_From_All_Time_High                   0
Percent_Difference_200_Day_Moving_Average    0
24_Hour_Percent_Change                       0
7_Day_Percent_Change                         0
30_Day_Percent_Change                        0
Annualized_Return                            0
YTD_Return                                   0
2024_Return                                  0
2023_Return                                  0
2022_Return                                  0
2021_Return                                  0
2020_Return                                  0
Annualized_Volatility                        0
Sharpe_Ratio                                 0
Beta                                         0
Company_Name                                 0
Market_Cap                                   0
Sector                                       0
Industry     

In [10]:
assert df_enriched_stock_data.isna().sum().sum() == 0, "DataFrame contains NaN values"

In [11]:
df_enriched_stock_data.to_csv('/Users/ani/Projects/6_stock_portfolio_recommendation/data/enriched_stock_data.csv', index=False)
print("Enriched stock data saved to 'data/enriched_stock_data.csv'")

Enriched stock data saved to 'data/enriched_stock_data.csv'
